# Design Flowsheet for NGCC + SOEC with Steam Integration

This flowsheet example provides an off-design model for integration of an SOEC for hydrogen production with an NGCC.  The NGCC nominally produces 650 MW net, and includeds 97% CO2 capture.  A detailed SOEC unit model is used here.

## Import Required Modules

In [1]:
import os
import math
import numpy as np
import pytest
from IPython.core.display import SVG
import pyomo.environ as pyo
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes
import idaes.core.util.scaling as iscale
import idaes.core.util as iutil
import ngcc_soec_costing ### changed name of costing module
import ngcc_soec

## Make Output Directories

This notebook can produce a large number of output files.  To make it easier to manage, some subdirectories are used to orginize output.  This ensures that the drectories exist.

In [2]:
def make_directory(path):
    """Make a directory if it doesn't exist"""
    try:
        os.mkdir(path)
    except FileExistsError:
        pass
    
make_directory("data")
make_directory("data_pfds")
make_directory("data_tabulated")

## Set Global Solver Options

Setting global solver options applies them to any solver created subsequently, including the ones used for initialization.  The user scaling option disables Ipopt's automatic scaling and allows it to use user provided variable scaling.

In [3]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
idaes.cfg.ipopt.options.linear_solver = "ma57"
idaes.cfg.ipopt.options.OF_ma57_automatic_scaling = "yes"
idaes.cfg.ipopt.options.ma57_pivtol = 1e-5
idaes.cfg.ipopt.options.ma57_pivtolmax = 0.1
idaes.cfg.ipopt.options.max_iter = 200
solver = pyo.SolverFactory("ipopt")

## Create and Initialize the NGCC + SOEC Model

In [ ]:
m = pyo.ConcreteModel()
m.fs = ngcc_soec.NgccSoecFlowsheet(default={"dynamic":False})
iscale.calculate_scaling_factors(m)
m.fs.initialize(
    load_from="ngcc_soec_init.json.gz",
    save_to="ngcc_soec_init.json.gz"
)

m.fs.ngcc.fuel_cost.fix(4.42)
m.fs.ngcc.cap_specific_reboiler_duty.fix(2.4e6)
m.fs.ngcc.cap_fraction.fix(0.97)

res = solver.solve(m, tee=True)

ngcc_soec_costing.add_results_for_costing(m)  ### need to calculate more results to be used by power plant costing
ngcc_soec_costing.get_ngcc_soec_capital_cost(m) ### changed name of costing function

res = solver.solve(m, tee=True)

2022-07-13 12:28:00 [INFO] idaes.init.fs.ngcc: NGCC load initial from ngcc_init.json.gz
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec: SOEC Initialization Starting
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec.sweep_compressor.control_volume.properties_in: Starting initialization
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec.sweep_compressor.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec.sweep_compressor.control_volume.properties_out: Starting initialization
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec.sweep_compressor.control_volume.properties_out: Property initialization: optimal - Optimal Solution Found.
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec.sweep_compressor.control_volume.properties_out: Property package initialization: optimal - Optimal Solution Found.
2022-07-13 12:28:00 [INFO] idaes.init.fs.soec.sweep_compressor.properties_isentropic: Starting initialization
2022-07-13 12:28:00 [INFO

In [ ]:
def display_pfd():
    print("\n\nGas Turbine Section\n")
    display(SVG(m.fs.ngcc.gt.write_pfd()))
    print("\n\nHRSG Section\n")
    display(SVG(m.fs.ngcc.hrsg.write_pfd()))
    print("\n\nSteam Turbine Section\n")
    display(SVG(m.fs.ngcc.st.write_pfd()))
    print("\n\nSOEC Section\n")
    display(SVG(m.fs.soec.write_pfd()))

In [ ]:
m.fs.ngcc.net_power_mw.display()
m.fs.ngcc.st.steam_turbine.throttle_valve[1].deltaP.display()

In [ ]:
decision_vars = []
def make_decision_var(v, lb, ub):
    v.unfix()
    v.setlb(lb)
    v.setub(ub)
    decision_vars.append(v)

# Add constraints for optimization
m.fs.ngcc.st.steam_turbine.throttle_valve[1].deltaP.setub(-1e5)
m.fs.soec.sweep_recycle_split.mixed_state[0].mole_frac_comp["O2"].setub(0.35)
m.fs.soec.feed_recycle_split.mixed_state[0].mole_frac_comp["H2O"].setlb(0.20)
m.fs.soec.sweep_recycle_split.mixed_state[0].temperature.setub(1030)
m.fs.soec.feed_recycle_split.mixed_state[0].temperature.setub(1030)
m.fs.ngcc.st.steam_turbine_lp_split.mixed_state[0].pressure.setlb(2.9e5)

@m.fs.Constraint(m.fs.time)
def makeup_water_constraint(b, t):
    return m.fs.soec.water_pump.inlet.flow_mol[t] == m.fs.soec.feed_hx01.tube_inlet.flow_mol[t]

# make sure the delta T on the hydrogen side is 40K or less (it's 
# squared so one constraint covers both positive and negative delta T)
@m.fs.Constraint(m.fs.time)
def delta_T_h_constraint(b, t):
    return (
        m.fs.soec.feed_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.feed_recycle_split.mixed_state[t].temperature
    )**2/100 <= 16

# make sure the delta T on the oxygen side is 40K or less
@m.fs.Constraint(m.fs.time)
def delta_T_o_constraint(b, t):
    return (
        m.fs.soec.sweep_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.sweep_recycle_split.mixed_state[t].temperature
    )**2/100 <= 16

# make sure the oxygen inlet and hydrogen outlet are 40K or less apart
@m.fs.Constraint(m.fs.time)
def delta_T_1_constraint(b, t):
    return (
        m.fs.soec.sweep_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.feed_recycle_split.mixed_state[t].temperature
    )**2/100 <= 16

# make sure the hydrogen inlet and oxygen outlet are 40K or less apart
@m.fs.Constraint(m.fs.time)
def delta_T_2_constraint(b, t):
    return (
        m.fs.soec.feed_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.sweep_recycle_split.mixed_state[t].temperature
    )**2/100 <= 16

@m.fs.Constraint(m.fs.time)
def average_current_density_constraint(b, t):
    return m.fs.soec.soec_module.solid_oxide_cell.average_current_density[0]/1000 >= -8

m.fs.soec.water_pump.inlet.flow_mol.unfix()
make_decision_var(m.fs.soec.sweep_recycle_split.split_fraction[0, "out"], 0.50, 0.97)
make_decision_var(m.fs.soec.feed_recycle_split.split_fraction[0, "out"], 0.50, 0.98) 
make_decision_var(m.fs.soec.water_split.split_fraction[0, "outlet1"], 0.3, 0.7)
make_decision_var(m.fs.soec.sweep_compressor.inlet.flow_mol[0], 100, 8000)
make_decision_var(m.fs.soec.soec_module.potential_cell[0], 1.26, 1.38)
make_decision_var(m.fs.soec.feed_heater.control_volume.properties_out[0].temperature, 900, 1020)
make_decision_var(m.fs.soec.sweep_heater.control_volume.properties_out[0].temperature, 900, 1020)
make_decision_var(m.fs.soec.sweep_heater.control_volume.properties_out[0].temperature, 900, 1020)
make_decision_var(m.fs.soec.soec_module.number_cells, 1e6, 2e6)
make_decision_var(m.fs.soec.sweep_hx.area, 2000, 7000)
make_decision_var(m.fs.soec.feed_hx01.area, 2000, 7000)
make_decision_var(m.fs.soec.water_heater01.area, 2000, 7000)
make_decision_var(m.fs.soec.water_heater02.area, 2000, 7000)

m.fs.obj_design = pyo.Objective(
    expr=(
        m.fs.ngcc.total_variable_cost_rate[0] + 
        m.fs.soec.variable_makeup_water_cost[0] +
        (m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6 * 100 +
        m.fs.costing.total_TPC*1e6 * 1.093 * 0.0707 * 1.341 / 365 / 24 # anualized cap. + fixed O&M for SOEC part
    )/1000
)

In [ ]:
res = solver.solve(m, tee=True)

In [ ]:
display_pfd()

In [ ]:
# Print freed decision vars
for v in decision_vars:
    print(f"{v} {pyo.value(v)}")

print("\n")
print("------------------------------------------")
print("Fixed Costs for Optimized Design")
print("------------------------------------------")

tpc = pyo.value(m.fs.costing.total_TPC*1e6)  ### the main costing block is now under fs
tasc = pyo.value(m.fs.costing.total_TPC*1e6)*1.21*1.093
ac = tasc*0.0707
print(f"TPC = {tpc}")
print(f"TASC = {tasc}")
print(f"Annualized TASC (MM$/yr) = {ac/1e6}")

# Parameters
n_op = 8
hourly_rate = 38.50
labor_burden = 30

# Fixed O&M components
annual_op_labor = n_op * hourly_rate * 8760 * (1 + labor_burden/100)/1e6
maint_labor = tpc * 0.4 * 0.019 / 1e6
maint_material = tpc * 0.6 * 0.019 / 1e6
admin_labor = 0.25*(annual_op_labor + maint_labor)
prop_tax_ins = 0.02*tpc/1e6
soec_replace = pyo.value(4.2765*m.fs.soec.soec_module.number_cells)/1e6

print("Fixed O&M Costs")
print(f"annual_op_labor (MM$/yr) = {annual_op_labor}")
print(f"maint_labor (MM$/yr) = {maint_labor}")
print(f"maint_material (MM$/yr) = {maint_material}")
print(f"admin_labor (MM$/yr) = {admin_labor}")
print(f"prop_tax_ins (MM$/yr) = {prop_tax_ins}")
print(f"soec_replace (MM$/yr) = {soec_replace}")
total_fixed = annual_op_labor + maint_labor + maint_material + admin_labor + prop_tax_ins + soec_replace
print(f"Annualized Fixed O&M = {total_fixed}")

print("\n")
print("------------------------------------------")
print("SOEC TPC Breakdown")
print("------------------------------------------")

ngcc_soec_costing.display_capital_costs(m) ### added new function to display the costs from power plant costing

In [ ]:
# lock in the design vars
m.fs.soec.soec_module.number_cells.fix()
m.fs.soec.sweep_hx.area.fix()
m.fs.soec.feed_hx01.area.fix()
m.fs.soec.water_heater01.area.fix()
m.fs.soec.water_heater02.area.fix()

m.fs.obj_design.deactivate()
m.fs.obj_op = pyo.Objective(
    expr=(
        m.fs.ngcc.total_variable_cost_rate[0] + 
        m.fs.soec.variable_makeup_water_cost[0] +
        (m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6 * 100
    )
)
res = solver.solve(m, tee=True)

In [ ]:
m.fs.ngcc.gt.write_pfd(fname="data_pfds/gt_soec_base.svg")
m.fs.ngcc.hrsg.write_pfd(fname="data_pfds/hrsg_soec_base.svg")
m.fs.ngcc.st.write_pfd(fname="data_pfds/st_soec_base.svg")
m.fs.soec.write_pfd(fname="data_pfds/soec_soec_base.svg")
display_pfd()

m.fs.ngcc.gt.streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_gt.csv")
m.fs.ngcc.st.steam_streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_st.csv")
m.fs.ngcc.hrsg.steam_streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_hrsg_steam.csv")
m.fs.ngcc.hrsg.flue_gas_streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_hrsg_gas.csv")
m.fs.soec.streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_soec.csv")

In [ ]:
run_samples = True
df = None
if run_samples:
    import pandas as pd
    gt_powers = np.linspace(480., 305., int((480. - 305.)/5.0) + 1).tolist()
    hp_powers = np.linspace(5, 0.75, int((5. - 0.75)/0.25) + 1).tolist()
    
    df = pd.DataFrame(columns=m.fs.tags_output.table_heading())
    i = 1
    for gp in gt_powers:
        gpstr = str(math.ceil(gp))
        m.fs.ngcc.gt.gt_power.fix(-gp*1e6)
        save_to_last = None
        for hp in hp_powers:
            if hp > 5.001 - 0.25*(480 - gp)/30:
                continue
            m.fs.soec.hydrogen_product_rate.fix(hp)
            hpstr = str(math.ceil(hp*1000))
            save_to = f"data/ngcc_soec_{gpstr}_{hpstr}.json.gz"
            print(save_to)
            if os.path.exists(save_to):
                iutil.from_json(m, fname=save_to, wts=iutil.StoreSpec(suffix=False))                    
                if save_to_last is None:
                    save_to_last = save_to
            else:
                res = solver.solve(m, tee=True)
                if not pyo.check_optimal_termination(res):
                    print("Fail")
                    break
                if save_to_last is None:
                    save_to_last = save_to
                iutil.to_json(m, fname=save_to)
            print(f"Overall net power: {-pyo.value(m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6} MW")
            #for v in decision_vars:
            #    print(f"{v} {pyo.value(v)}")
            df.loc[i] = m.fs.tags_output.table_row(numeric=True)
            i += 1
        # go to the closest part of the start of the next series.
        iutil.from_json(m, fname=save_to_last, wts=iutil.StoreSpec(suffix=False))

In [ ]:
if df is not None:
    df.to_csv("data_tabulated/ngcc_soec.csv")